In [1]:
# import statements
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
# assessing the .tsv file that holds the kinematic data ({x,y,z} coordinates for each marker)
file_3D = pd.read_csv('Trial0001_static.tsv', delimiter='\t', skiprows=10)

file_3D.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Frame             129 non-null    int64  
 1   Time              129 non-null    float64
 2   1 - Head X        129 non-null    float64
 3   1 - Head Y        129 non-null    float64
 4   1 - Head Z        129 non-null    float64
 5   2 - L_Shoulder X  129 non-null    float64
 6   2 - L_Shoulder Y  129 non-null    float64
 7   2 - L_Shoulder Z  129 non-null    float64
 8   3 - L_Elbow X     129 non-null    float64
 9   3 - L_Elbow Y     129 non-null    float64
 10  3 - L_Elbow Z     129 non-null    float64
 11  4 - L_Wrist X     129 non-null    float64
 12  4 - L_Wrist Y     129 non-null    float64
 13  4 - L_Wrist Z     129 non-null    float64
 14  5 - R_Shoulder X  129 non-null    float64
 15  5 - R_Shoulder Y  129 non-null    float64
 16  5 - R_Shoulder Z  129 non-null    float64
 1

In [3]:
file_3D.head()

,Frame,Time,1 - Head X,1 - Head Y,1 - Head Z,2 - L_Shoulder X,2 - L_Shoulder Y,2 - L_Shoulder Z,3 - L_Elbow X,3 - L_Elbow Y,...,17 - R_Heel X,17 - R_Heel Y,17 - R_Heel Z,18 - R_Meta_V X,18 - R_Meta_V Y,18 - R_Meta_V Z,19 - R_Toe_II X,19 - R_Toe_II Y,19 - R_Toe_II Z,Unnamed: 59
0,119,1.18,341.204,299.258,1814.116,317.832,510.946,1488.631,301.931,562.535,...,236.033,251.099,22.720,399.810,167.430,23.294,499.254,204.631,20.032,NaN
1,120,1.19,341.176,299.248,1814.007,317.728,510.856,1488.634,301.864,562.602,...,236.033,251.101,22.707,399.861,167.511,23.330,499.280,204.685,20.000,NaN
2,121,1.20,341.355,299.344,1814.062,317.745,510.905,1488.553,301.766,562.595,...,235.992,251.097,22.660,399.755,167.580,23.196,499.174,204.691,19.999,NaN
3,122,1.21,341.324,299.194,1814.076,317.803,510.987,1488.576,301.684,562.639,...,235.944,251.112,22.760,399.724,167.554,23.251,499.276,204.633,19.994,NaN
4,123,1.22,341.528,299.236,1814.164,317.754,510.966,1488.644,301.454,562.650,...,235.973,251.127,22.785,399.857,167.493,23.250,499.313,204.650,19.970,NaN


In [4]:
# Project 3D data onto 2D plane (dropping y-axis information)
columns_to_drop = [col for col in file_3D.columns if ' Y' in col]

# Drop the columns from the DataFrame
# The `axis=1` tells pandas to drop columns, not rows.
file_2D = file_3D.drop(columns=columns_to_drop, axis=1)
file_2D.info()

selected_columns = file_2D.columns[2:]

for col in selected_columns:
    file_2D[col] = file_2D[col]/1000

print(file_2D)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Frame             129 non-null    int64  
 1   Time              129 non-null    float64
 2   1 - Head X        129 non-null    float64
 3   1 - Head Z        129 non-null    float64
 4   2 - L_Shoulder X  129 non-null    float64
 5   2 - L_Shoulder Z  129 non-null    float64
 6   3 - L_Elbow X     129 non-null    float64
 7   3 - L_Elbow Z     129 non-null    float64
 8   4 - L_Wrist X     129 non-null    float64
 9   4 - L_Wrist Z     129 non-null    float64
 10  5 - R_Shoulder X  129 non-null    float64
 11  5 - R_Shoulder Z  129 non-null    float64
 12  6 - R_Elbow X     129 non-null    float64
 13  6 - R_Elbow Z     129 non-null    float64
 14  7 - R_Wrist X     129 non-null    float64
 15  7 - R_Wrist Z     129 non-null    float64
 16  8 - L_Hip X       129 non-null    float64
 1

In [5]:
def create_coordinate_columns(df):
    """
    Transforms separate 'Marker X' and 'Marker Z' columns into
    a single 'Marker_Coord' column of (X, Z) tuples.
    """
    # Get all column names as a list
    all_columns = df.columns.tolist()

    # Identify unique marker bases (e.g., '1 - Head', '2 - L_Shoulder')
    marker_bases = set()
    for col in all_columns:
        if col.endswith(' X') or col.endswith(' Z'):
            # The base is everything up to the last space,
            # e.g., '1 - Head' from '1 - Head X'
            base = col[:-2].strip()
            marker_bases.add(base)

    # Create the new DataFrame for coordinates
    coord_df = pd.DataFrame()

    # Iterate through each identified marker base and create the coordinate column
    for base in sorted(list(marker_bases)):
        x_col = f'{base} X'
        z_col = f'{base} Z'
        coord_col = f'{base}_Coord'

        # Ensure both X and Z columns exist for this base
        if x_col in df.columns and z_col in df.columns:
            coord_df[coord_col] = df.apply(
                lambda row: (row[x_col], row[z_col]),
                axis=1
            )

    return coord_df

# 3. Create the new DataFrame with coordinate tuples
coord_df = create_coordinate_columns(file_2D)

print("\nDataFrame with only Coordinate Tuples:\n", coord_df)


DataFrame with only Coordinate Tuples:
                       1 - Head_Coord  \
0               (0.341204, 1.814116)   
1     (0.341176, 1.8140070000000001)   
2     (0.341355, 1.8140619999999998)   
3               (0.341324, 1.814076)   
4     (0.341528, 1.8141639999999999)   
..                               ...   
124              (0.35137, 1.813242)   
125             (0.351522, 1.813235)   
126             (0.351537, 1.813234)   
127  (0.35159500000000005, 1.813254)   
128             (0.351764, 1.813226)   

                             10 - L_Ankle_Coord  \
0               (0.28139400000000003, 0.064485)   
1                          (0.281313, 0.064558)   
2                          (0.281311, 0.064496)   
3                          (0.281435, 0.064399)   
4               (0.281423, 0.06448000000000001)   
..                                          ...   
124  (0.28136700000000003, 0.06446299999999999)   
125                          (0.281289, 0.0644)   
126             (0.

In [6]:
def calculate_segment_com(df, proximal_col, distal_col, com_ratio):
    """
    Calculates the Center of Mass (CoM) coordinates for a body segment across all frames.

    Args:
        df (pd.DataFrame): The DataFrame containing the coordinate columns.
        proximal_col (str): Column name for the proximal joint coordinates (e.g., 'Hip_Coord').
        distal_col (str): Column name for the distal joint coordinates (e.g., 'Knee_Coord').
        com_ratio (float): The segment-specific CoM location as a fraction (e.g., 0.433).

    Returns:
        pd.Series: A Series of (X_CoM, Z_CoM) tuples for the new CoM location.
    """
    # 1. Extract X and Z components for both joints
    x1 = df[proximal_col].str[0] # X-coord of proximal joint
    z1 = df[proximal_col].str[1] # Z-coord of proximal joint

    x2 = df[distal_col].str[0]   # X-coord of distal joint
    z2 = df[distal_col].str[1]   # Z-coord of distal joint

    # 2. Apply the Biomechanics CoM formula (vectorized):
    # C = P1 + (P2 - P1) * L_ratio

    # Calculate X component of CoM
    x_com = x1 + (x2 - x1) * com_ratio

    # Calculate Z component of CoM
    z_com = z1 + (z2 - z1) * com_ratio

    # 3. Combine the results back into a Series of (X, Z) tuples
    return pd.Series(list(zip(x_com, z_com)))


In [7]:
# Composing Dataframe 'Points' for biomechanical model
num_rows = len(file_2D)

# Define the column names
column_names = [f'P{i}' for i in range(1, 16)] # Generates ['P1', 'P2', ..., 'P14']

# 2. Create an array of NaNs with the correct shape (Rows x Columns)
# The data type is set to float (dtype='float64')
data = np.full((num_rows, len(column_names)), np.nan)

# 3. Create the final DataFrame
points = pd.DataFrame(
    data,
    columns=column_names
)

# --- Display the result ---
print("\n--- New Target DataFrame Layout ---")
print(f"Shape: {points.shape} (Rows x Columns)")
print(points.head()) # Shows the first few rows

# Determining and storing shoulder midpoint data to points dataframe
points['P2'] = calculate_segment_com(
    coord_df,
    proximal_col='2 - L_Shoulder_Coord',
    distal_col='5 - R_Shoulder_Coord',
    com_ratio=0.5
)

points['P7'] = calculate_segment_com(
    coord_df,
    proximal_col='8 - L_Hip_Coord',
    distal_col='14 - R_Hip_Coord',
    com_ratio=0.5
)

print(points.head())


--- New Target DataFrame Layout ---
Shape: (129, 15) (Rows x Columns)
   P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  P11  P12  P13  P14  P15
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
   P1                                P2  P3  P4  P5  P6  \
0 NaN   (0.3229205, 1.4893230000000002) NaN NaN NaN NaN   
1 NaN   (0.3228465, 1.4892975000000002) NaN NaN NaN NaN   
2 NaN             (0.3228525, 1.489244) NaN NaN NaN NaN   
3 NaN   (0.32285600000000003, 1.489235) NaN NaN NaN NaN   
4 NaN  (0.32285050000000004, 1.4892805) NaN NaN NaN NaN   

                                 P7  P8  P9  P10  P11  P12  P13  P14  P15  
0              (0.327061, 0.932978) NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  (0.32708499

In [8]:
points['P1'] = coord_df['1 - Head_Coord']
points['P3'] = coord_df['6 - R_Elbow_Coord']
points['P4'] = coord_df['7 - R_Wrist_Coord']
points['P5'] = coord_df['3 - L_Elbow_Coord']
points['P6'] = coord_df['4 - L_Wrist_Coord']
points['P8'] = coord_df['15 - R_Knee_Coord']
points['P9'] = coord_df['16 - R_Ankle_Coord']
points['P10'] = coord_df['18 - R_Meta_V_Coord']
points['P11'] = coord_df['19 - R_Toe_II_Coord']
points['P12'] = coord_df['9 - L_Knee_Coord']
points['P13'] = coord_df['10 - L_Ankle_Coord']
points['P14'] = coord_df['12 - L_Meta_V_Coord']
points['P15'] = coord_df['13 - L_Toe_II_Coord']

print(points)

                                  P1                                P2  \
0               (0.341204, 1.814116)   (0.3229205, 1.4893230000000002)   
1     (0.341176, 1.8140070000000001)   (0.3228465, 1.4892975000000002)   
2     (0.341355, 1.8140619999999998)             (0.3228525, 1.489244)   
3               (0.341324, 1.814076)   (0.32285600000000003, 1.489235)   
4     (0.341528, 1.8141639999999999)  (0.32285050000000004, 1.4892805)   
..                               ...                               ...   
124              (0.35137, 1.813242)             (0.328691, 1.4893805)   
125             (0.351522, 1.813235)             (0.3287815, 1.489318)   
126             (0.351537, 1.813234)            (0.3287525, 1.4893065)   
127  (0.35159500000000005, 1.813254)             (0.328847, 1.4893585)   
128             (0.351764, 1.813226)                (0.328786, 1.4894)   

                                 P3  \
0    (0.290221, 1.1239970000000001)   
1     (0.29019, 1.124032999999999

In [9]:
def calculate_average_distance_from_coords(coord_df, marker1_coord_col, marker2_coord_col):
    """
    Calculates the Euclidean distance between two marker coordinate columns.

    Args:
        coord_df (pd.DataFrame): The DataFrame containing columns of (X, Z) tuples.
        marker1_coord_col (str): The full name of the first coordinate column (e.g., '1 - Head_Coord').
        marker2_coord_col (str): The full name of the second coordinate column (e.g., '2 - L_Shoulder_Coord').

    Returns:
        float: The average Euclidean distance between the two markers.
    """

    # 1. Use the NumPy vectorized approach again
    # This is done by extracting the X (index 0) and Z (index 1) components
    # from the tuples in the pandas Series.

    # Vectorized extraction of X and Z components
    x1 = coord_df[marker1_coord_col].str[0]
    z1 = coord_df[marker1_coord_col].str[1]

    x2 = coord_df[marker2_coord_col].str[0]
    z2 = coord_df[marker2_coord_col].str[1]

    # 2. Calculate the Euclidean Distance (vectorized)
    distances = np.sqrt((x2 - x1)**2 + (z2 - z1)**2)

    # 3. Calculate and return the average
    return distances.mean()

In [10]:
# Right side of body
avg_right_upper_arm_length = calculate_average_distance_from_coords(points, 'P2','P3')
avg_right_forearm_length = calculate_average_distance_from_coords(points,'P3','P4')

avg_right_thigh_length = calculate_average_distance_from_coords(points,'P7','P8')
avg_right_shank_length = calculate_average_distance_from_coords(points,'P8','P9')
avg_right_midfoot_length = calculate_average_distance_from_coords(points,'P9','P10')
avg_right_midfoot_length = calculate_average_distance_from_coords(points,'P10','P11')

# Left side of body
avg_left_upper_arm_length = calculate_average_distance_from_coords(points, 'P2','P5')
avg_left_forearm_length = calculate_average_distance_from_coords(points,'P5','P6')

avg_left_thigh_length = calculate_average_distance_from_coords(points,'P7','P12')
avg_left_shank_length = calculate_average_distance_from_coords(points,'P12','P13')
avg_left_midfoot_length = calculate_average_distance_from_coords(points,'P13','P14')
avg_left_midfoot_length = calculate_average_distance_from_coords(points,'P14','P15')

# Middle part of the body
avg_neck_length = calculate_average_distance_from_coords(points,'P1','P2')
avg_trunk_length = calculate_average_distance_from_coords(points,'P2','P7')

In [11]:
print(f"\nAverage right upper arm length: {avg_right_upper_arm_length:.2f}")
print(f"\nAverage right forearm length: {avg_right_forearm_length:.2f}")
print(f"\nAverage trunk length: {avg_trunk_length:.2f}")
print(f"\nAverage neck length: {avg_neck_length:.2f}")
print(f"\nAverage right thigh length: {avg_right_thigh_length:.2f}")
print(f"\nAverage shank length: {avg_right_shank_length:.2f}")


Average right upper arm length: 0.37

Average right forearm length: 0.25

Average trunk length: 0.56

Average neck length: 0.33

Average right thigh length: 0.42

Average shank length: 0.45


In [ ]:
def calculate_segment_kinematics(df, proximal_col, distal_col, com_ratio, segment_name):
    """
    Calculates the Segment Center of Mass (x, z) and Orientation (theta)
    for all time steps. Returns a Series of (x_com, z_com, theta) tuples.
    """
    # 1. Extract X and Z components using a list comprehension (robust method for tuples)
    # The 'map' function is often cleaner than list comprehensions when the column holds tuples

    # x1/z1 = Proximal (P1) coords
    # x2/z2 = Distal (P2) coords
    x1 = df[proximal_col].apply(lambda t: t[0])
    z1 = df[proximal_col].apply(lambda t: t[1])
    x2 = df[distal_col].apply(lambda t: t[0])
    z2 = df[distal_col].apply(lambda t: t[1])

    # 2. Calculate CoM (Position)
    # C = P1 + (P2 - P1) * L_ratio
    x_com = x1 + (x2 - x1) * com_ratio
    z_com = z1 + (z2 - z1) * com_ratio

    # 3. Calculate Orientation (Angle, theta)
    # Segment Vector V = P2 - P1 (Distal - Proximal)
    V_x = x2 - x1
    V_z = z2 - z1

    # Calculate angle in radians using atan2(Vz, Vx)
    # arctan2 is vectorized and handles all quadrants correctly
    theta = np.arctan2(V_z, V_x)

    # 4. Combine results into a Series of (x_com, z_com, theta) tuples
    kinematics_series = pd.Series(list(zip(x_com, z_com, theta)), name=segment_name)

    return kinematics_series